1. [X] (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?


2. [X] (2 балла) Реализуйте один из предложенных вами способов.


3. [X] (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)


4. [X] (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).


5. [X] (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.


- [X] jupyter-ноутбук + укажите, какие данные брали — или положите их тоже в репозиторий.

# Идеи как найти упоминания

1. Посчитать частотные существительные по всему корпусу,с помоцью w2v и найти наиболее близкие к выделенным существительным. *Плюсы*: получается большой словарь синонимов. *Минусы*: надо использвать готовую модель (в которой может не быть некоторых слов) или обуать свою (при большом размере корпуса это может быть долго); если руками не фильтровать существительные, то туда попадет много всего лишнего.


2. **То же самое, но пересечь частотные слова с леммами из поддерева данной категории в WordNet** *Плюсы*: все собирается автоматически. *Минусы*: какого-то количества слов точно не будет в итоговом словаре 


3. Взять самые большие отзывы/склеить отзывы на 1 товар (по мета данным) и применить к ним методы выделения ключевых слов. *Плюсы*: больших отзывов достаточно много, может неплохо сработать. *Минусы*: нужно сводить отзывы с метаданными; возможно будут выделяться не NE, а просто устойчивые словосочетания, типичные для жанра отзывов.


4. Использовать готовый NER. *Плюсы*: почти все уже готово. *Минусы*: "поймаются" только то, что содержит название бренда.


5. Посмотреть на пересечение Текста и Саммари. Плюсы и минусы сложно прикинуть.

In [49]:
import json
import gzip
from tqdm.auto import tqdm

import nltk
from matplotlib import pyplot as plt
from collections import Counter

import pickle
import string
from nltk.corpus import stopwords
stops = stopwords.words('english')

Использую `Musical Instruments` sample dataset [отсюда](https://nijianmo.github.io/amazon/index.html), потому что я начала делать задание до того, как заметила, что в задании обновили ссылку на данные.

In [145]:
data = []
with gzip.open('Musical_Instruments_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

231392


In [146]:
data[0]

{'overall': 5.0,
 'verified': True,
 'reviewTime': '10 30, 2016',
 'reviewerID': 'A3FO5AKVTFRCRJ',
 'asin': '0739079891',
 'reviewerName': 'francisco',
 'reviewText': "It's good for beginners",
 'summary': 'Five Stars',
 'unixReviewTime': 1477785600}

In [147]:
empty = []
for i, d in enumerate(data):
    try:
        _ = d['reviewText']
    except KeyError:
        empty.append(i)
        
print(empty)

[16840, 16999, 17465, 17627, 25914, 30139, 31309, 32299, 33575, 37233, 39787, 42966, 52475, 54780, 58474, 68900, 79377, 86939, 95705, 104343, 109024, 127662, 153286, 166639, 166658, 168957, 171927, 173470, 173968, 181465, 198929, 199454, 201808, 202113, 202378, 203860, 204207, 204410, 209119, 212036, 217238, 217584, 219970, 223213, 225944, 229448, 229479, 229572]


In [148]:
corpus = [d['reviewText'] for i, d in enumerate(data) if i not in empty]

## Получение слов из WordNet'a

In [66]:
from nltk.corpus import wordnet as wn

In [87]:
musical_instrument = wn.synset('musical_instrument.n.01')

In [92]:
musical_instrument.hyponyms()

[Synset('barrel_organ.n.01'),
 Synset('bass.n.07'),
 Synset('calliope.n.02'),
 Synset('electronic_instrument.n.01'),
 Synset('jew's_harp.n.01'),
 Synset('keyboard_instrument.n.01'),
 Synset('music_box.n.01'),
 Synset('percussion_instrument.n.01'),
 Synset('stringed_instrument.n.01'),
 Synset('wind_instrument.n.01')]

In [101]:
instruments = []
syns = [musical_instrument]

while syns:
    for syn in syns:
        instruments.append(syn)
        syns.remove(syn)
        syns.extend(syn.hyponyms())

instr_list = list(set(instruments))
len(instr_list)

164

In [107]:
musical_instruments = []
for i in instr_list:
    musical_instruments.extend(i.lemma_names()) 

len(musical_instruments)

299

In [109]:
print(musical_instruments)

['banjo', 'pennywhistle', 'tin_whistle', 'whistle', 'cittern', 'cithern', 'cither', 'citole', 'gittern', 'organ_stop', 'tenor_drum', 'tom-tom', "oboe_d'amore", 'bandoneon', 'clavier', 'Klavier', 'saxhorn', 'baby_grand', 'baby_grand_piano', 'parlor_grand', 'parlor_grand_piano', 'parlour_grand', 'parlour_grand_piano', 'virginal', 'pair_of_virginals', 'bass_guitar', 'bass_horn', 'sousaphone', 'tuba', 'tambourine', 'snare_drum', 'snare', 'side_drum', 'nose_flute', 'koto', 'dulcimer', 'vox_angelica', 'voix_celeste', 'mandola', 'clarion', 'post_horn', 'tabor', 'tabour', 'chanter', 'melody_pipe', 'concertina', 'American_organ', 'drone', 'drone_pipe', 'bourdon', 'hornpipe', 'pibgorn', 'stockhorn', 'free-reed', 'zill', 'violin', 'fiddle', 'dulcimer', 'viola_da_gamba', 'gamba', 'bass_viol', 'viola_da_braccio', 'trigon', 'percussion_instrument', 'percussive_instrument', 'bass', 'triangle', 'calliope', 'steam_organ', 'viola', 'marimba', 'xylophone', 'grand_piano', 'grand', 'flugelhorn', 'fluegelho

## Обработка корпуса

In [128]:
import spacy

In [133]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [150]:
corpus_processed = []

for text in tqdm(corpus):
    corpus_processed.append(nlp(text))

In [151]:
# with open('corpus_processed.pickle', 'wb') as f:
#     pickle.dump(corpus_processed, f)

In [240]:
for token in corpus_processed[0]:
    print(token.text, token.lemma_, token.pos_, token.is_stop)

It -PRON- PRON True
's be AUX True
good good ADJ False
for for ADP True
beginners beginner NOUN False


### Смотрю на самые частотные слова (и существительные)

In [165]:
lemmas = []

for doc in tqdm(corpus_processed):
    for token in doc:
        lemmas.append((token.lemma_, token.pos_))
        
len(lemmas)

15629086

In [170]:
freq = Counter(lemmas)

In [183]:
top_words_tagged = []
top_nouns_tagged = []
for w_t, f in freq.most_common(5000):
    w, t = w_t
    if w not in stops and w not in string.punctuation:
        top_words_tagged.append(w)
        if t == 'NOUN':
            top_nouns_tagged.append(w)
len(top_words_tagged)

4784

In [184]:
len(top_nouns_tagged)

1874

In [308]:
top_nouns_tagged[:100]

['guitar',
 'string',
 'sound',
 'price',
 'pedal',
 'quality',
 'time',
 'one',
 'product',
 'amp',
 'cable',
 'tone',
 'thing',
 'year',
 'case',
 'pick',
 'stand',
 'bass',
 'lot',
 'way',
 'instrument',
 'bit',
 'strap',
 'mic',
 'tuner',
 'problem',
 'set',
 'power',
 'review',
 'effect',
 'music',
 'unit',
 'end',
 'speaker',
 'money',
 'volume',
 'neck',
 'issue',
 'bag',
 'box',
 'work',
 'use',
 'size',
 'star',
 'microphone',
 'light',
 'drum',
 'job',
 'noise',
 'pickup',
 'day',
 'brand',
 'head',
 'side',
 'battery',
 'gig',
 'knob',
 'hand',
 'board',
 'metal',
 'part',
 'control',
 'tune',
 'player',
 'keyboard',
 'studio',
 'recording',
 'month',
 'note',
 'tube',
 'plastic',
 'item',
 'device',
 'color',
 'tuning',
 'difference',
 'level',
 'finger',
 'hole',
 'piece',
 'feature',
 'switch',
 'model',
 'couple',
 'room',
 'input',
 'people',
 'screw',
 'deal',
 'button',
 'setting',
 'capo',
 'bridge',
 'home',
 'band',
 'range',
 'line',
 'type',
 'style',
 'point']

Пересекаю частотные слова и список лемм из WN

In [203]:
instr = list(set([i for i in top_words_tagged if i in musical_instruments]))
print(instr)

['wind',
 'tambourine',
 'sax',
 'recorder',
 'bass',
 'harp',
 'flute',
 'chime',
 'mandolin',
 'ukulele',
 'clarinet',
 'snare',
 'banjo',
 'synthesizer',
 'dulcimer',
 'string',
 'reed',
 'pipe',
 'fiddle',
 'instrument',
 'cello',
 'drum',
 'baritone',
 'grand',
 'horn',
 'cymbal',
 'brass',
 'organ',
 'trombone',
 'harmonica',
 'saxophone',
 'upright',
 'bongo',
 'piano',
 'triangle',
 'whistle',
 'trumpet',
 'guitar',
 'violin',
 'wood',
 'uke',
 'bell']

# Получение колокаций

In [309]:
from spacy.matcher import Matcher

### смотрю на упонинания инструментов и существительных на -or/-er с имеами собственными

In [253]:
left_propn_nouner = [{'POS': 'ADJ'},
                    {'POS': 'PROPN'},
                    {'POS': 'NOUN', 'LEMMA': {"REGEX": "[eo]r$"}}]

left_propn_instr = [{'POS': 'ADJ'},
                   {'POS': 'PROPN'},
                   {"LEMMA": {"IN": instr}}]

ne_matcher = Matcher(nlp.vocab)
ne_matcher.add("left_PROPN_er", None, left_propn_nouner)
ne_matcher.add("left_PROPN_instr", None, left_propn_instr)


propn_matches = []

for doc in tqdm(corpus_processed):
    matches = ne_matcher(doc)

    for match_id, start, end in matches:
        # string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        propn_matches.append(span.text)
        
len(propn_matches)

2036

In [254]:
propn_matches[:100]

['inferior Firebrand uke',
 'loyal yamaha consumer',
 'new Switchcraft connector',
 'multiple Hosa adapters',
 'second Hosa guitar',
 'cheap XLR connectors',
 'dedicated mic splitter',
 'old Rickenbacker guitar',
 'powered PA loudspeaker',
 'old Sony reciever',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'little Zenergy chimes',
 'old Sony reciever',
 'real Shure instrument',
 'sized Phillips screwdriver',
 'other Casio power',
 'standard Technics rubber',
 '1U decibel meter',
 'wireless mic transmitter',
 'big bell horn',
 "pricier D'Addario strings",
 'high e string',
 'old Sony speaker',
 'nice Fender bell',
 'old So

### ищу 4 основных шаблона с прилагательными
т.к. именно они выражают оценку/харатеристику - то, что нужно в отчете

коллокации работают сильно хуже, см. ниже

In [234]:
adj_instr = [{'POS': 'ADJ'}, 
             {"LEMMA": {"IN": instr}}]
doubleadj_instr = [{'POS': 'ADJ'}, 
                   {'POS': 'ADJ'},
                   {"LEMMA": {"IN": instr}}]
adv_adj_instr = [{'POS': 'ADV'}, 
                   {'POS': 'ADJ'},
                   {"LEMMA": {"IN": instr}}]
instr_be_smth = [{"LEMMA": {"IN": instr}},
                 {'LEMMA': 'be'}, 
                 {'POS': {"IN": ['ADJ', 'NOUN']}}]

matcher = Matcher(nlp.vocab)
matcher.add("ADJ_instr", None, adj_instr)
matcher.add("doubleADJ_instr", None, doubleadj_instr)
matcher.add("ADV_ADJ_instr", None, adv_adj_instr)
matcher.add("instr_be_smth", None, instr_be_smth)

In [242]:
all_matches = []

for doc in tqdm(corpus_processed):
    matches = matcher(doc)

    for match_id, start, end in matches:
        # string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        all_matches.append(span.text)
        
len(all_matches)

62704

In [243]:
matches_c = Counter(all_matches)
sorted_matches = matches_c.most_common()

### Получается вот такой отчет

In [244]:
reviews = {}
for ins in instr:
    reviews[ins] = []

for m, f in sorted_matches:
    for ins in instr:
        if ins in m:
            reviews[ins].append({'phrase': m,
                                 'count': f})

In [315]:
from pprint import pprint

In [326]:
for i in ['guitar', 'drum', 'sax', 'recorder', 'string']:
    pprint(reviews[i][:20])
    print('\n')

[{'count': 2552, 'phrase': 'acoustic guitar'},
 {'count': 1802, 'phrase': 'electric guitar'},
 {'count': 846, 'phrase': 'acoustic guitars'},
 {'count': 748, 'phrase': 'electric guitars'},
 {'count': 645, 'phrase': 'classical guitar'},
 {'count': 432, 'phrase': 'other guitars'},
 {'count': 415, 'phrase': 'new guitar'},
 {'count': 372, 'phrase': 'great guitar'},
 {'count': 252, 'phrase': 'other guitar'},
 {'count': 232, 'phrase': 'good guitar'},
 {'count': 225, 'phrase': 'cheap guitar'},
 {'count': 205, 'phrase': 'nice guitar'},
 {'count': 200, 'phrase': 'first guitar'},
 {'count': 189, 'phrase': 'favorite guitar'},
 {'count': 185, 'phrase': 'different guitars'},
 {'count': 177, 'phrase': 'little guitar'},
 {'count': 173, 'phrase': 'best guitar'},
 {'count': 171, 'phrase': 'Great guitar'},
 {'count': 168, 'phrase': 'local guitar'},
 {'count': 151, 'phrase': 'expensive guitar'}]


[{'count': 98, 'phrase': 'electronic drum'},
 {'count': 56, 'phrase': 'electronic drums'},
 {'count': 49, 'ph

Сохраняю его в .json файл

In [310]:
import json

In [322]:
with open('reviews.json', 'w') as f:
    json.dump(reviews, f, indent=2)

## Биграммы

Не очень понимаю зачем так грубо брать контекст, но вот он

In [225]:
left_context = [{'IS_PUNCT': False},
                {"LEMMA": {"IN": instr}}]
right_context = [{"LEMMA": {"IN": instr}},
                 {'IS_PUNCT': False}]

context_matcher = Matcher(nlp.vocab)
context_matcher.add("left_context", None, left_context)
context_matcher.add("right_context", None, right_context)

all_context = []

for doc in tqdm(corpus_processed):
    matches = context_matcher(doc)

    for match_id, start, end in matches:
        # string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        all_context.append(span.text)
        
len(all_context)

355482

In [230]:
contexts = Counter(all_context)
contexts.most_common(10)

[('the guitar', 8934),
 ('the strings', 6238),
 ('my guitar', 5708),
 ('these strings', 4944),
 ('a guitar', 3965),
 ('guitar and', 3635),
 ('your guitar', 3476),
 ('this guitar', 2939),
 ('strings are', 2818),
 ('acoustic guitar', 2559)]

In [255]:
from nltk.collocations import *

In [278]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [301]:
ngrams = []
for c in all_context:
    n = 0
    words = c.lower().split()
    for w in words:
        if w in stops:
            n += 1
    if n == 0:
        ngrams.append(words)
    
ngrams[:10]

[['stringed', 'instrument'],
 ['stringed', 'instrument'],
 ['strings', 'really'],
 ['uke', 'held'],
 ['laminated', 'instrument'],
 ['unknown', 'wood'],
 ['soprano', 'ukes'],
 ['ukes', 'generally'],
 ['specialty', 'ukulele'],
 ['ukulele', 'shop']]

In [302]:
finder = BigramCollocationFinder.from_documents(ngrams)

In [303]:
ll_scores = finder.score_ngrams(bigram_measures.likelihood_ratio)
ll_scores[:20]

[(('guitar', 'guitar'), 16480.719464277965),
 (('strings', 'guitar'), 10444.070288719724),
 (('strings', 'strings'), 6831.153462323573),
 (('guitar', 'bass'), 4569.310104011272),
 (('acoustic', 'guitar'), 4365.147320890128),
 (('great', 'strings'), 4207.626859105092),
 (('12', 'string'), 3602.89323876078),
 (('strings', 'string'), 3512.8326357940614),
 (('e', 'string'), 3446.696792048923),
 (('string', 'strings'), 3340.165883991948),
 (('guitar', 'string'), 3075.379070791222),
 (('guitar', 'strap'), 3072.4768184643426),
 (("d'addario", 'strings'), 3039.273111044713),
 (('strings', 'bass'), 3011.300421064483),
 (('electric', 'guitar'), 2985.426071593826),
 (('drum', 'machine'), 2981.495152853903),
 (('guitar', 'guitars'), 2838.4472031379128),
 (('drum', 'kit'), 2390.2428881263527),
 (('g', 'string'), 2314.294717138254),
 (('drum', 'set'), 2295.4041980755655)]

In [305]:
pmi_scores = finder.score_ngrams(bigram_measures.pmi)
pmi_scores[:20]

[(('ta', 'uking'), 16.76051638283185),
 (('drummed', 'sense'), 16.345478883553007),
 (("'ve", 'piped'), 14.76051638283185),
 (('dulcimers', 'singe'), 14.438588287944487),
 (('acute', 'triangles'), 14.345478883553007),
 (('enormous', 'triangles'), 14.345478883553007),
 (('raven', 'triangles'), 14.345478883553007),
 (('triangles', 'oh'), 14.345478883553007),
 (('throat', 'trumpets'), 14.258016042302668),
 (('squeaky', 'fiddles'), 14.175553882110695),
 (('carry', 'bongoes'), 14.023550788665645),
 (('pono', 'baritones'), 13.88604726491571),
 (('hour', 'piping'), 13.853625787223333),
 (('moon', 'tambourines'), 13.821916927495995),
 (('harmonium', 'organs'), 13.701622693778283),
 (('harpsichord', 'organs'), 13.701622693778283),
 (('johannus', 'organs'), 13.701622693778283),
 (('vital', 'organs'), 13.701622693778283),
 (('whistling', 'galore'), 13.645039165411914),
 (('chiming', 'highs'), 13.590591381389537)]

In [306]:
dice_scores = finder.score_ngrams(bigram_measures.dice)
dice_scores[:20]

[(('drummed', 'sense'), 0.5),
 (('ta', 'uking'), 0.5),
 (('pitch', 'pipe'), 0.43425076452599387),
 (('mountain', 'dulcimer'), 0.3865546218487395),
 (('zenergy', 'chimes'), 0.3008849557522124),
 (('chimes', 'sat'), 0.29310344827586204),
 (("'ve", 'piped'), 0.2857142857142857),
 (('hour', 'piping'), 0.2857142857142857),
 (('pono', 'baritones'), 0.26666666666666666),
 (('bongo', 'cajon'), 0.24193548387096775),
 (('alto', 'sax'), 0.23926940639269406),
 (('tin', 'whistle'), 0.23295454545454544),
 (('hammered', 'dulcimer'), 0.22),
 (('note', 'chimes'), 0.21935483870967742),
 (('wind', 'screen'), 0.21871476888387825),
 (('analog', 'synthesizers'), 0.2127659574468085),
 (('wind', 'noise'), 0.21272365805168986),
 (('20', 'harmonicas'), 0.1875),
 (('mph', 'winds'), 0.17647058823529413),
 (('grand', 'piano'), 0.17510090034150885)]

Мне кажется что PMI дает наиболее хороший результат в выделении NE, но для "отчета" для них надо еще раз взять контекст (оценочные прилагательные).

*Анна Полянская, БКЛ171*